In [28]:
import pandas as pd
import numpy as np

In [29]:
df_schools  = pd.read_csv("raw_data/schools_complete.csv")
df_students = pd.read_csv("raw_data/students_complete.csv")

## District Summary

In [30]:
passing_rate = 0.5*(len(df_students[df_students.math_score>65])/len(df_students) + len(df_students[df_students.reading_score>65])/len(df_students))
df_summary = pd.DataFrame({"Summary Item":
                           ['Total schools', 'Total students','Total budget','Average Math Score',
                           'Average Reading Score','Percentage of student passing Math (>65)',
                            'Percentage of student passing Reading (>65)','Overall passing rate'],
                          "Data":['{:02d}'.format(len(df_schools)),len(df_students),'$'+'{:,}'.format(df_schools['budget'].sum()),
                                  df_students['math_score'].mean(),df_students['reading_score'].mean(),
                                  '{:4.2f}'.format(100*len(df_students[df_students.math_score>65])/len(df_students))+"%",
                                  '{:4.2f}'.format(100*len(df_students[df_students.reading_score>65])/len(df_students))+"%",
                                  '{:4.2f}'.format(100*passing_rate)+"%"]})

In [31]:
df_summary[['Summary Item','Data']]

,Summary Item,Data
0,Total schools,15
1,Total students,39170
2,Total budget,"$24,649,428"
3,Average Math Score,78.9854
4,Average Reading Score,81.8778
5,Percentage of student passing Math (>65),83.11%
6,Percentage of student passing Reading (>65),94.26%
7,Overall passing rate,88.69%


## School Summary

In [32]:
df_schools = df_schools.sort_values('name').reset_index(drop=True)

In [33]:
df_student_count = pd.DataFrame(df_students.groupby('school').count()['name'])
df_student_count = df_student_count.reset_index(drop=False)
df_student_count.columns = ['name','Number_of_students']

In [34]:
df_schools_2= df_schools.set_index('name').join(df_student_count.set_index('name'))
df_schools_2 = df_schools_2.reset_index(drop=False)
df_schools_2['Budget per student']=df_schools_2.budget/df_schools_2.Number_of_students
df_mean_math = pd.DataFrame(df_students.groupby('school')['math_score'].mean()).reset_index(drop=False)
df_mean_math.columns = ['name','mean_math_score']
df_schools_3 = df_schools_2.set_index('name').join(df_mean_math.set_index('name'))
df_schools_3 = df_schools_3.reset_index(drop=False)

In [51]:
df_mean_reading = pd.DataFrame(df_students.groupby('school')['reading_score'].mean()).reset_index(drop=False)
df_mean_reading.columns = ['name','mean_reading_score']
df_schools_4 = df_schools_3.set_index('name').join(df_mean_reading.set_index('name'))
df_schools_4 = df_schools_4.reset_index(drop=False)

In [52]:
# Need the percentage of students that are passing math in each school
# 1. Get the number of students in each school
#    df_schools_4['Number_of_students']
# 2. In each of those school, get the number of students that have passed math 
number_students_passing = df_students[df_students.math_score>65].groupby('school').count()
df_schools_4['Percentage_passing_math']=100*number_students_passing.reset_index()['math_score']/df_schools_4['Number_of_students']

In [53]:
number_students_passing_reading = df_students[df_students.reading_score>65].groupby('school').count()
df_schools_4['Percentage_passing_reading']=100*number_students_passing_reading.reset_index()['reading_score']/df_schools_4['Number_of_students']

In [54]:
df_schools_4['Overall Passing Rate'] = 0.5*(df_schools_4['Percentage_passing_reading']+df_schools_4['Percentage_passing_math'])

In [112]:
list_of_items =['name','type','Number_of_students','Budget per student','mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']
schools_summary = df_schools_4.copy()
schools_summary = schools_summary[list_of_items]
schools_summary['Percentage_passing_math'] = schools_summary['Percentage_passing_math'].map('{:4.2f}%'.format)
schools_summary['Percentage_passing_reading'] = schools_summary['Percentage_passing_reading'].map('{:4.2f}%'.format)
schools_summary['Overall Passing Rate'] = schools_summary['Overall Passing Rate'].map('{:4.2f}%'.format)
schools_summary['Budget per student'] = schools_summary['Budget per student'].map('${:,}'.format)
schools_summary

,name,type,Number_of_students,Budget per student,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
0,Bailey High School,District,4976,$628.0,77.048432,81.033963,75.60%,91.90%,83.75%
1,Cabrera High School,Charter,1858,$582.0,83.061895,83.975780,100.00%,100.00%,100.00%
2,Figueroa High School,District,2949,$639.0,76.711767,81.158020,74.87%,91.90%,83.38%
3,Ford High School,District,2739,$644.0,77.102592,80.746258,76.20%,90.80%,83.50%
4,Griffin High School,Charter,1468,$625.0,83.351499,83.816757,100.00%,100.00%,100.00%
5,Hernandez High School,District,4635,$652.0,77.289752,80.934412,75.47%,91.46%,83.46%
6,Holden High School,Charter,427,$581.0,83.803279,83.814988,100.00%,100.00%,100.00%
7,Huang High School,District,2917,$655.0,76.629414,81.182722,75.28%,91.77%,83.53%
8,Johnson High School,District,4761,$650.0,77.072464,80.966394,75.38%,92.06%,83.72%
9,Pena High School,Charter,962,$609.0,83.839917,84.044699,100.00%,100.00%,100.00%


## Top Performing Schools by Passing Rate

In [113]:
df_schools_top = df_schools_4.sort_values('Overall Passing Rate',ascending=False)
df_schools_top = df_schools_top[list_of_items]
df_schools_top['Percentage_passing_math'] = df_schools_top['Percentage_passing_math'].map('{:4.2f}%'.format)
df_schools_top['Percentage_passing_reading'] = df_schools_top['Percentage_passing_reading'].map('{:4.2f}%'.format)
df_schools_top['Overall Passing Rate'] = df_schools_top['Overall Passing Rate'].map('{:4.2f}%'.format)
df_schools_top['Budget per student'] = df_schools_top['Budget per student'].map('${:,}'.format)
df_schools_top.head(5)

,name,type,Number_of_students,Budget per student,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
1,Cabrera High School,Charter,1858,$582.0,83.061895,83.975780,100.00%,100.00%,100.00%
4,Griffin High School,Charter,1468,$625.0,83.351499,83.816757,100.00%,100.00%,100.00%
6,Holden High School,Charter,427,$581.0,83.803279,83.814988,100.00%,100.00%,100.00%
9,Pena High School,Charter,962,$609.0,83.839917,84.044699,100.00%,100.00%,100.00%
11,Shelton High School,Charter,1761,$600.0,83.359455,83.725724,100.00%,100.00%,100.00%


## Bottom Performing Schools by Passing Rate

In [114]:
df_schools_bottom = df_schools_4.sort_values('Overall Passing Rate')
df_schools_bottom = df_schools_bottom[list_of_items]
df_schools_bottom['Percentage_passing_math'] = df_schools_bottom['Percentage_passing_math'].map('{:4.2f}%'.format)
df_schools_bottom['Percentage_passing_reading'] = df_schools_bottom['Percentage_passing_reading'].map('{:4.2f}%'.format)
df_schools_bottom['Overall Passing Rate'] = df_schools_bottom['Overall Passing Rate'].map('{:4.2f}%'.format)
df_schools_bottom['Budget per student'] = df_schools_bottom['Budget per student'].map('${:,}'.format)
df_schools_bottom.head(5)

,name,type,Number_of_students,Budget per student,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
2,Figueroa High School,District,2949,$639.0,76.711767,81.158020,74.87%,91.90%,83.38%
5,Hernandez High School,District,4635,$652.0,77.289752,80.934412,75.47%,91.46%,83.46%
3,Ford High School,District,2739,$644.0,77.102592,80.746258,76.20%,90.80%,83.50%
7,Huang High School,District,2917,$655.0,76.629414,81.182722,75.28%,91.77%,83.53%
10,Rodriguez High School,District,3999,$637.0,76.842711,80.744686,75.54%,91.52%,83.53%


## Math scores by grade

In [115]:
pd.DataFrame(df_students.groupby('grade').mean()['math_score']).reset_index()

,grade,math_score
0,10th,78.941483
1,11th,79.083548
2,12th,78.993164
3,9th,78.935659


## Reading scores by grade

In [116]:
pd.DataFrame(df_students.groupby('grade').mean()['reading_score']).reset_index()

,grade,reading_score
0,10th,81.874410
1,11th,81.885714
2,12th,81.819851
3,9th,81.914358


## Scores by school size

In [82]:
print("Max budget per student: $" + str(df_schools_4['Budget per student'].max()))
print("Min budget per student: $" + str(df_schools_4['Budget per student'].min()))

Max budget per student: $655.0
Min budget per student: $578.0


In [86]:
bins=[575.,595.,615.,635.,655.]
group_bins = ["\$575 to \$595","\$595 to \$615","\$615 to \$635","\$635 to \$655"]

In [87]:
df_schools_4['View Budget per student groups']=pd.cut(df_schools_4['Budget per student'].map(np.float),bins,labels=group_bins)

In [103]:
df_schools_scores_budget = df_schools_4.groupby('View Budget per student groups').mean()[['mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']]
df_schools_scores_budget['Percentage_passing_math'] = df_schools_scores_budget['Percentage_passing_math'].map('{:5.2f}%'.format)
df_schools_scores_budget['Percentage_passing_reading'] = df_schools_scores_budget['Percentage_passing_reading'].map('{:5.2f}%'.format)
df_schools_scores_budget['Overall Passing Rate'] = df_schools_scores_budget['Overall Passing Rate'].map('{:5.2f}%'.format)
df_schools_scores_budget

,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
View Budget per student groups,,,,,
\$575 to \$595,83.455399,83.933814,100.00%,100.00%,100.00%
\$595 to \$615,83.599686,83.885211,100.00%,100.00%,100.00%
\$615 to \$635,80.199966,82.425360,87.80%,95.95%,91.88%
\$635 to \$655,77.866721,81.368774,78.96%,92.79%,85.88%


In [104]:
print("Max number of student: " + str(df_schools_4['size'].max()))
print("Min number of student: " + str(df_schools_4['size'].min()))

Max number of student: 4976
Min number of student: 427


In [105]:
bins=[400,2000,3500,5000]
group_bins = ["Small (400 to 2000)","Medium (2000 to 3500)","Large (3500 to 5000)"]

In [106]:
df_schools_4['View number of student groups']=pd.cut(df_schools_4['size'],bins,labels=group_bins)

In [107]:
df_schools_scores_size = df_schools_4.groupby('View number of student groups').mean()[['mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']]
df_schools_scores_size['Percentage_passing_math'] = df_schools_scores_size['Percentage_passing_math'].map('{:5.2f}%'.format)
df_schools_scores_size['Percentage_passing_reading'] = df_schools_scores_size['Percentage_passing_reading'].map('{:5.2f}%'.format)
df_schools_scores_size['Overall Passing Rate'] = df_schools_scores_size['Overall Passing Rate'].map('{:5.2f}%'.format)
df_schools_scores_size


,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
View number of student groups,,,,,
Small (400 to 2000),83.502373,83.883125,100.00%,100.00%,100.00%
Medium (2000 to 3500),78.429493,81.769122,81.59%,93.62%,87.60%
Large (3500 to 5000),77.063340,80.919864,75.50%,91.74%,83.62%


In [109]:
df_schools_scores_type = df_schools_4.groupby('type').mean()[['mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']]
df_schools_scores_type['Percentage_passing_math'] = df_schools_scores_type['Percentage_passing_math'].map('{:5.2f}%'.format)
df_schools_scores_type['Percentage_passing_reading'] = df_schools_scores_type['Percentage_passing_reading'].map('{:5.2f}%'.format)
df_schools_scores_type['Overall Passing Rate'] = df_schools_scores_type['Overall Passing Rate'].map('{:5.2f}%'.format)
df_schools_scores_type

,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,100.00%,100.00%,100.00%
District,76.956733,80.966636,75.48%,91.63%,83.55%
